In [26]:
%load_ext autoreload
%autoreload 2

In [27]:
import sys
sys.path.insert(0, '..//src/')

from datetime import datetime
import requests
import pandas as pd
from project_settings import settings


In [34]:

headers = {
  'Accept': 'application/json'
}
API_ENDPOINT = "https://api.carbonintensity.org.uk/intensity/{}/{}/"

# Function to fetch data for a given date range
def fetch_data_for_date_range(from_datetime:datetime, to_datetime:datetime):
    formatted_from_date = from_datetime.strftime("%Y-%m-%dT%H:%MZ")
    formatted_to_date = to_datetime.strftime("%Y-%m-%dT%H:%MZ")
    url = API_ENDPOINT.format(formatted_from_date, formatted_to_date)
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data from {formatted_from_date} to {formatted_to_date}")
        return None
    

def convert_json_to_df(results):
    carbon_intensity = pd.DataFrame(results['data'])
    carbon_intensity['from'] = pd.to_datetime(carbon_intensity['from'], format='%Y-%m-%dT%H:%MZ')
    carbon_intensity['to'] = pd.to_datetime(carbon_intensity['to'], format='%Y-%m-%dT%H:%MZ')
    carbon_intensity = pd.concat([carbon_intensity[['from', 'to']], carbon_intensity['intensity'].apply(pd.Series)], axis=1)
    return carbon_intensity 

frames = []
for temp_datetime in pd.date_range(datetime(2023, 1, 1), datetime(2024, 7,1), freq='d'):
  print(temp_datetime)
  year = temp_datetime.year
  day = temp_datetime.day
  month = temp_datetime.month
  results = fetch_data_for_date_range(datetime(year, month, day), datetime(year, month, day, 23, 30))
  if results:
    if len(results['data'])>0:
      temp_df = convert_json_to_df(results)
      frames.append(temp_df)


2023-01-01 00:00:00
2023-01-02 00:00:00
2023-01-03 00:00:00
2023-01-04 00:00:00
2023-01-05 00:00:00
2023-01-06 00:00:00
2023-01-07 00:00:00
2023-01-08 00:00:00
2023-01-09 00:00:00
2023-01-10 00:00:00
2023-01-11 00:00:00
2023-01-12 00:00:00
2023-01-13 00:00:00
2023-01-14 00:00:00
2023-01-15 00:00:00
2023-01-16 00:00:00
2023-01-17 00:00:00
2023-01-18 00:00:00
2023-01-19 00:00:00
2023-01-20 00:00:00
2023-01-21 00:00:00
2023-01-22 00:00:00
2023-01-23 00:00:00
2023-01-24 00:00:00
2023-01-25 00:00:00
2023-01-26 00:00:00
2023-01-27 00:00:00
2023-01-28 00:00:00
2023-01-29 00:00:00
2023-01-30 00:00:00
2023-01-31 00:00:00
2023-02-01 00:00:00
2023-02-02 00:00:00
2023-02-03 00:00:00
2023-02-04 00:00:00
2023-02-05 00:00:00
2023-02-06 00:00:00
2023-02-07 00:00:00
2023-02-08 00:00:00
2023-02-09 00:00:00
2023-02-10 00:00:00
2023-02-11 00:00:00
2023-02-12 00:00:00
2023-02-13 00:00:00
2023-02-14 00:00:00
2023-02-15 00:00:00
2023-02-16 00:00:00
2023-02-17 00:00:00
2023-02-18 00:00:00
2023-02-19 00:00:00


In [43]:
carbon_intensity_df = pd.concat(frames)
carbon_intensity_df.set_index('from', inplace=True)
carbon_intensity_df = carbon_intensity_df[['actual']]
carbon_intensity_df.columns = ['carbon_intensity (gCO2/kWh)']
carbon_intensity_df.to_csv(settings.PATH_SAVE_DATA/'carbon_intensity.csv')